# SRL
Whether it's a short data processing app, and api layer, or a full blown distributed system, many systems follow an activity paradigm. Once you see it, you can't unsee it:
1) An agent, or a process, works on some data
2) There is a temporary or ephemeral nature about "its" data. Like a player, an instance, a stock market algorithm, or even an api call. But within that moment, that local information in critical.
3) The ephemeral data should pass something on to the global context
4) The global context, objects, rankings, accounts, trades, need to change
5) The world updates, and with a new timestep, we expect notifications about all the worlds activity to flow
6) The agent has views, and filters / searches the worlds information

This loop is in many systems. Over time I have learned that much of my development activity is centered around "scaling" this exact loop:

1) in the smallest case, I write a one file simple client / server system.
2) Then i code up a front end, and I need to host it
3) Suddenly I need to scale the system across a million different nodes.

Moving from 1-2 is hard. It's about the same difficulty to move from 2-3. BUT, approaching every problem with (3) will increase the time of every project. If only there was a thin framework that one could adopt at (1) that would allow you to scale to (3) if that ever becomes needed.

Welcome to activity. Activity is the least amount of infastructre needed at (1) to ensure your application can seamlessly scale to (3) with minimal adjustments. It enforces patterns of object orientation (encapsulation), and statless communication via an event-subscriber model. But, thankfully, it mostly works using function decorators in python. This means, the (1) version can be as simple as a few function calls

Activity is built on processingNetwork, a generic processing engine that simplifies functional programming (making it feel procedural, if you squint at it.)

If the script you are working on will never be (3) then it's safe to say you should just code it. If it's worth 20 minutes of planning, then refactor onto activity right away, and you will be treated later,

In [1]:


# Agent
# ---
# Strategy (Strategy)
# Action Tracker (Order Tracker)

# System Template
# ----------------
# Local Environment (Account Log)
# World Environment (Broker View)

# Pace
# ---------
# TimeEvent
# Clock

# View
# -----------------
# TimeseriesEvent (MarketDataEmitter / Order Assignment Event)
# Filter (Filter Model)



In [28]:
import sys
sys.path.append('../../')
from flex import flex
from processingNetwork import ProcessingNode,ProcessingNetwork,QuickInputNode,QuickProcessingNode,PersistanceNetwork

import uuid
class EnvironmentManager():
    def FindObject(feature,context):
        messages = []
        if 'object_id' in feature:
            if feature['object_id'] in context['objects'].keys():
                messages.append (context['objects'][feature['object_id']])
            else:
                messages.append (context['objects'])            
            return messages
        return messages.append ({'error':'no objects found'})

    
    
    def UpdateObjectRequest(feature,context):
        if 'objects' not in context:
            context['objects'] = {}
            context['create_count'] = 0
        objects = context['objects']
        messages = []
        for action in feature:
            if action == 'create':
                context['create_count'] = context['create_count'] + 1
                try:
                    obj = flex()
                    obj.x = feature[action]['x']
                    obj.y = feature[action]['y']
                    obj.z = feature[action]['z']
                    obj.owner_id = feature[action]['owner_id']
                    if obj.object_id in objects:
                        objects[obj.object_id].update(obj)
                    else:
                        obj.object_id = str(uuid.uuid4())
                        objects[obj.object_id] = obj
                    feature[action]['status'] = 'true'
                    messages.append(obj)
                except Exception as e:
                    feature[action]['status'] = 'false'
                    feature[action]['error'] = str(e)
                    messages.append(feature)
                print(context['objects'])
        return messages

feature={'create':{'name':'charles','x':0,'y':0,'z':0,'owner_id':10}}


messages = EnvironmentManager.UpdateObjectRequest(feature={'create':{'name':'charles','x':0,'y':0,'z':0,'owner_id':10}},context={})
print(messages )

#messages = EnvironmentManager.FindObject()


{}
[{'create': {'name': 'charles', 'x': 0, 'y': 0, 'z': 0, 'owner_id': 10, 'status': 'false', 'error': "'object_id'"}}]


In [33]:
from conf.conf import conf
conn = {'mongo_server':conf.get( "jef_mongo_host", "" ) ,
        'mongo_port':conf.get( "jef_mongo_port",  ),
        'mongo_username':conf.get( "jef_mongo_username", "" ),
        'mongo_password':conf.get( "jef_mongo_password", "" ),
        'mongo_authSource':conf.get( "jef_mongo_authSource", "" ),
        'mongo_database':conf.get( "jef_mongo_object_database", "srl" ),
        'mongo_collection':conf.get( "srl_mongo_experiments_collection", "srl_persistance" ) ,}       

nodes = flex()

#data_packet = {'data':feature}
#nodes['layer1'] = QuickInputNode(EnvironmentManager.UpdateObjectRequest,('data',))
#pn = PersistanceNetwork(networkDef=nodes,mongo_connection=conn,serverKey={'server_id':1})
#data_out = pn.process(data_packet)
#import pprint
#pprint.pprint(data_packet)

data_packet = {'find':{'object_id':'89b4a7be-3fed-4f77-b8cc-c914c9dadc8f'}}
data_packet = {'find':{'object_id':None}}
nodes['layer1'] = QuickInputNode(EnvironmentManager.FindObject,('find',))
pn = PersistanceNetwork(networkDef=nodes,mongo_connection=conn,serverKey={'server_id':1})
data_out = pn.process(data_packet)
import pprint
# Process a result
pprint.pprint(data_packet)


{'find': {'object_id': None},
 'layer1': [{'2b9c4130-f1fe-4cdb-8a56-24c3f87119b8': {'count': 2,
                                                      'object_id': '2b9c4130-f1fe-4cdb-8a56-24c3f87119b8',
                                                      'owner_id': 10,
                                                      'x': 0,
                                                      'y': 0,
                                                      'z': 0},
             '54c73c94-279f-4c0e-a41a-9fbc381e9879': {'count': 3,
                                                      'object_id': '54c73c94-279f-4c0e-a41a-9fbc381e9879',
                                                      'owner_id': 10,
                                                      'x': 0,
                                                      'y': 0,
                                                      'z': 0},
             '717c7665-9614-4e5a-a68b-cb9b06f2f3fb': {'count': 5,
                                            

This is *almost* serializable. In fact, if we got rid of these class refrences, it would be possible to serialize everything about this experiment. To support serialization, the code can't live in a notebook -- we need to give it a home. The syntax for the serializable version is below, as an example: